In [1]:
import random

Alice_input = 1  # Alice's input bit
Bob_input = 0    # Bob's input bit

# Making the Shares
def share(secret: int) -> tuple[int, int]:
    # Secret share a bit 'secret' between Alice and Bob.
    x_b = random.randint(0, 1)  # Bob's share
    x_a = secret ^ x_b           # Alice's share
    return (x_a, x_b)

# Reconstructing the secret from shares
def reconstruct(share: tuple[int, int]) -> int:
    # Reconstruct the bit from a sharing [x] = (xA, xB).
    x_a, x_b = share
    return x_a ^ x_b

alice_share = share(Alice_input)
bob_share = share(Bob_input)
print("Alice input:")
print(Alice_input)
print("Bob input:")
print(Bob_input)

Alice input:
1
Bob input:
0


In [2]:
# Individual Gates
def xor_const(share: tuple[int, int], const: int) -> tuple[int, int]:
    # const = 1 is the NOT gate
    x_a, x_b = share
    z_a = x_a ^ const
    z_b = x_b
    return (z_a, z_b)

print(reconstruct(xor_const(alice_share, 0)))
print(reconstruct(xor_const(bob_share, 0)))

def and_const(share: tuple[int, int], const: int) -> tuple[int, int]:
    # const = 0, zeros the gate
    x_a, x_b = share
    z_a = const * x_a
    z_b = const * x_b
    return (z_a, z_b)

print(reconstruct(and_const(alice_share, 1)))
print(reconstruct(and_const(bob_share, 1)))

# Two-Wire Gates
def xor_gate(share1: tuple[int, int], share2: tuple[int, int]) -> tuple[int, int]:
    x_a, x_b = share1
    y_a, y_b = share2
    z_a = x_a ^ y_a
    z_b = x_b ^ y_b
    return (z_a, z_b)

print(reconstruct(xor_gate(alice_share, bob_share)))  # Expected output: 0


1
0
1
0
1


In [3]:
Alice_input = (1,1,1)  # Alice's input bit
Bob_input = (0,0,)    # Bob's input bit

# Dealer Gates
def dealer() -> tuple[tuple[int, int], tuple[int, int], tuple[int, int]]:
    # Dealer provides random bits u, v, w such that w = u * v
    u = random.randint(0, 1)
    v = random.randint(0, 1)
    w = u * v
    return share(u), share(v), share(w)

def and_gate(share1: tuple[int, int], share2: tuple[int, int]) -> tuple[int, int]:
    # u, v, w are from the dealer
    (x_a, x_b), (y_a, y_b) = share1, share2
    (u_a, u_b), (v_a, v_b), (w_a, w_b) = dealer()

    d = xor_gate((x_a, x_b), (u_a, u_b))
    e = xor_gate((y_a, y_b), (v_a, v_b))

    # d and e become public
    d = reconstruct(d)
    e = reconstruct(e)

    z_a = w_a ^ (e * x_a) ^ (d * y_a) ^ (d * e)
    z_b = w_b ^ (e * x_b) ^ (d * y_b)
    return (z_a, z_b)

print(and_gate(alice_share, bob_share))

def or_gate(share1: tuple[int, int], share2: tuple[int, int]) -> int:
    # Using De Morgan's law: x OR y = NOT(NOT x AND NOT y)
    not_x = xor_const(share1, 1)
    not_y = xor_const(share2, 1)
    and_not = and_gate(not_x, not_y)
    return xor_const(and_not, 1)


(0, 0)


In [9]:
Alice_input = (1,0,0)  # Alice's input bit
Bob_input = (0,1,0)    # Bob's input bit

alice_share = (share(Alice_input[0]), share(Alice_input[1]), share(Alice_input[2]))
bob_share = (share(Bob_input[0]), share(Bob_input[1]), share(Bob_input[2]))


# Least Significant Bit (LSB) represents Rh of the blood type
# 0 = Rh-, 1 = Rh+
# The Two Most Significant Bits (MSB) represents ABO of the blood type
# O = 00, A = 01, B = 10, AB = 11
# Example: A+ = 011, O- = 000, AB+ = 111

def blood_type_compatibility_tester(recipient: tuple[tuple[int, int], tuple[int, int], tuple[int, int]], donor: tuple[tuple[int, int], tuple[int, int], tuple[int, int]]) -> int:
  # test Rh compatibility
  Rh_compatibility = or_gate(xor_const(donor[2], 1), recipient[2])

  # test ABO compatibility
  # CHECK IF DONOR IS O
  donor_is_O = and_gate(xor_const(donor[0], 1), xor_const(donor[1], 1))

  #if reconstruct(donor_is_O) == 1:
  #  return reconstruct(and_gate(Rh_compatibility, donor_is_O))
  
  A_type = and_gate(donor[1], recipient[1])  # donor A and recipient A
  B_type = and_gate(donor[0], recipient[0])  # donor B and recipient B
  
  ABO_compatibility = or_gate(donor_is_O, or_gate(A_type, B_type))

  # final compatibility
  compatibility = and_gate(Rh_compatibility, ABO_compatibility)

  return reconstruct(compatibility)  # 1 if compatible, 0 if not

blood_type_compatibility_tester(alice_share, bob_share)


0